In [ ]:
#Imports
%reset -f
import subprocess 
import os as os 
import shutil

In [ ]:
simulations_dir = '/Users/alexlee/OneDrive - Imperial College London/Year 4/Masters/code/masters_3d/temp/training data/targets' #the folder where all the simulations sub-folders are

generalExport_dir = '/Users/alexlee/OneDrive - Imperial College London/Year 4/Masters/code/masters_3d/temp/training data/export'

nodalDisplacements_dir_1 = generalExport_dir + 'Displacements\\Disp1\\' #directory where the contour "displacement vector field" is exported
nodalDisplacements_dir_2 = generalExport_dir + '\\Displacements\\Disp2\\' #directory where the contour "displacement vector field" is exported
nodalDisplacements_dir_3 = generalExport_dir + '\\Displacements\\Disp3\\' #directory where the contour "displacement vector field" is exported
nodalDisplacements_dir_4 = generalExport_dir + '\\Displacements\\Disp4\\' #directory where the contour "displacement vector field" is exported

majorStrainExport_dir_3 = generalExport_dir + '\\MajorStrain\\MajorStrain3\\' #directory where the contour "MajorStrain" is exported
majorStrainExport_dir_4 = generalExport_dir + '\\MajorStrain\\MajorStrain4\\' #directory where the contour "MajorStrain" is exported
minorStrainExport_dir_3 = generalExport_dir + '\\MinorStrain\\MinorStrain3\\' #directory where the contour "MinorStrain" is exported
minorStrainExport_dir_4 = generalExport_dir + '\\MinorStrain\\MinorStrain4\\' #directory where the contour "MinorStrain" is exported

misesStressExport_dir_3 = generalExport_dir + '\\MisesStress\\MisesStress3\\' #directory where the contour "MisesStress" is exported
misesStressExport_dir_4 = generalExport_dir + '\\MisesStress\\MisesStress4\\' #directory where the contour "MisesStress" is exported

thinningExport_dir_3 = generalExport_dir + '\\Thinning\\Thinning3\\' #directory where the contour "Thinning" is exported
thinningExport_dir_4 = generalExport_dir + '\\Thinning\\Thinning4\\' #directory where the contour "Thinning" is exported

deformedMeshes_dir_3 = generalExport_dir + '\\DeformedMeshes\\DeformedMeshes3\\' #directory where the deformed meshes are exported
deformedMeshes_dir_4 = generalExport_dir + '\\DeformedMeshes\\DeformedMeshes4\\' #directory where the deformed meshes are exported

PAMExportScripts_dir = generalExport_dir + '\\ExportScripts\\'

#the following lines are commands to be printed into a .txt file, which then acts as a script to be read by PAM
#to add additional commands, record a script in PAM GUI and then grab those commands and place it here
str_ref_1 = """Set PoeDoc = PoeApp.OpenDocument($Simulation_file_i$) 
Set PoeView = PoeDoc.Get3DView(1)
PoeDoc.ActivateModule(POST,END)
PoeView.DisplayObjects(Blank,NO)
PoeView.DisplayContour(DISPLACEMENT_VECTOR)
PoeView.ExportContour($DISPLACEMENT_file_i$,conasc.exe)
PoeDoc.CloseDocument()
PoeApp.Exit()"""

str_ref_2 = """Set PoeDoc = PoeApp.OpenDocument($Simulation_file_i$) 
Set PoeView = PoeDoc.Get3DView(1)
PoeDoc.ActivateModule(POST,END)
PoeView.DisplayObjects(Blank,NO)
PoeView.DisplayContour(STRAIN_MAJOR /FIBER="MEMBRANE" /VALUE_TYPE="TRUE")
PoeView.ExportContour($MajorStrain_file_i$,conasc.exe)
PoeView.DisplayContour(STRAIN_MINOR /FIBER="MEMBRANE" /VALUE_TYPE="TRUE")
PoeView.ExportContour($MinorStrain_file_i$,conasc.exe)
PoeView.DisplayContour(STRESS_VON_MISES /FIBER="MEMBRANE")
PoeView.ExportContour($MisesStress_file_i$,conasc.exe)
PoeView.DisplayContour(THINNING /VALUE_TYPE="ENGINEER")
PoeView.ExportContour($Thinning_file_i$,conasc.exe)
PoeView.DisplayContour(DISPLACEMENT_VECTOR)
PoeView.ExportContour($DISPLACEMENT_file_i$,conasc.exe)
PoeDoc.ExportMesh($DEFORMED_MESH_file_i$,ficpam.exe)
PoeDoc.CloseDocument()
PoeApp.Exit()"""


In [ ]:
###########################
### EXPORT PAM CONTOURS ###
###########################

failedSamples = []
for simulation_folder in os.listdir(simulations_dir): #for all simulations
    
    if 'Sample_' not in simulation_folder: #only consider 'Simulation_' folders
        continue #move to the next for loop iteration if 'Simulation_' is not in the folder name - i.e. if its another folder

    i_str = simulation_folder.rsplit('_')[1] #get the number of the current simulation as a string
    
    #setup paths using the current simulation
    sample_path_1 = simulations_dir.replace('/','\\') + simulation_folder + '\\Backbone765_' + i_str + '_01_Gravity.pre'
    displacement_path_1 = nodalDisplacements_dir_1 + 'NodalDisplacements_' + i_str + '.asc'
    
    sample_path_2 = simulations_dir.replace('/','\\') + simulation_folder + '\\Backbone765_' + i_str + '_02_Closing.pre'
    displacement_path_2 = nodalDisplacements_dir_2 + 'NodalDisplacements_' + i_str + '.asc'
    
    sample_path_3 = simulations_dir.replace('/','\\') + simulation_folder + '\\Backbone765_' + i_str + '_03_Stamping.pre'
    majorStrain_path_3 = majorStrainExport_dir_3 + 'MajorStrain_' + i_str + '.asc'
    minorStrain_path_3 = minorStrainExport_dir_3 + 'MajorStrain_' + i_str + '.asc'
    misesStress_path_3 = misesStressExport_dir_3 + 'MisesStress_' + i_str + '.asc'
    thinning_path_3 = thinningExport_dir_3 + 'Thinning_' + i_str + '.asc'
    displacement_path_3 = nodalDisplacements_dir_3 + 'NodalDisplacements_' + i_str + '.asc'
    deformedMeshes_path_3 = deformedMeshes_dir_3 + 'DeformedMesh_' + i_str + '.pc'
    
    sample_path_4 = simulations_dir.replace('/','\\') + simulation_folder + '\\Backbone765_' + i_str + '_04_Springback.pre'
    majorStrain_path_4 = majorStrainExport_dir_4 + 'MajorStrain_' + i_str + '.asc'
    minorStrain_path_4 = minorStrainExport_dir_4 + 'MajorStrain_' + i_str + '.asc'
    misesStress_path_4 = misesStressExport_dir_4 + 'MisesStress_' + i_str + '.asc'
    thinning_path_4 = thinningExport_dir_4 + 'Thinning_' + i_str + '.asc'
    displacement_path_4 = nodalDisplacements_dir_4 + 'NodalDisplacements_' + i_str + '.asc'
    deformedMeshes_path_4 = deformedMeshes_dir_4 + 'DeformedMesh_' + i_str + '.pc'
    
    #check to see if the 'end' file exists - means the simulation sucessfully ended
    StampingFileName = 'Backbone765_' + i_str + '_04_Springback.end.res'
    sample_path_end_file = simulations_dir.replace('/','\\') + simulation_folder + '\\' + StampingFileName
    is_file = os.path.isfile(sample_path_end_file)

    if is_file: #if the end files exist (if the simulation has comlpeted sucessfully)
        
        #create and write to a .txt file which will be the commands that PAM will use to import (and automatically mesh) the CAD files into the tool editor
        #this file contains scriptable commands which were obtained by recording them in PAM using the GUI
        export_script_i_fn_1 = PAMExportScripts_dir + 'ExportScript1_' + i_str + '.txt'
        
        with  open(export_script_i_fn_1,'w') as export_script_i_f:
            #write the lines for PAM to read, using the above defined parameters, to import CAD (and mesh automatically) and build the tools
            str_i = str_ref_1.replace('$Simulation_file_i$',sample_path_1)
            str_i = str_i.replace('$DISPLACEMENT_file_i$',displacement_path_1)
            export_script_i_f.write(str_i)
        
        thinningContourExists = os.path.isfile(thinning_path_3)
        if thinningContourExists is not True: #only export the contours if they dont exist, if it does exists, don't generate
            
            #launch PAM to run its script to create the .mif mesh
            cmd='{} -play={}'.format(stamp_dn,export_script_i_fn_1)
            subprocess.call(cmd)
            
        print(simulation_folder, 'Stage I Completed')
        
        #create and write to a .txt file which will be the commands that PAM will use to import (and automatically mesh) the CAD files into the tool editor
        #this file contains scriptable commands which were obtained by recording them in PAM using the GUI
        export_script_i_fn_2 = PAMExportScripts_dir + 'ExportScript2_' + i_str + '.txt'
        
        with  open(export_script_i_fn_2,'w') as export_script_i_f:
            #write the lines for PAM to read, using the above defined parameters, to import CAD (and mesh automatically) and build the tools
            str_i = str_ref_1.replace('$Simulation_file_i$',sample_path_2)
            str_i = str_i.replace('$DISPLACEMENT_file_i$',displacement_path_2)
            export_script_i_f.write(str_i)
        
        thinningContourExists = os.path.isfile(thinning_path_3)
        if thinningContourExists is not True: #only export the contours if they dont exist, if it does exists, don't generate
            
            #launch PAM to run its script to create the .mif mesh
            cmd='{} -play={}'.format(stamp_dn,export_script_i_fn_2)
            subprocess.call(cmd)
            
        print(simulation_folder, 'Stage II Completed')
        
        #create and write to a .txt file which will be the commands that PAM will use to import (and automatically mesh) the CAD files into the tool editor
        #this file contains scriptable commands which were obtained by recording them in PAM using the GUI
        export_script_i_fn_3 = PAMExportScripts_dir + 'ExportScript3_' + i_str + '.txt'
        
        with  open(export_script_i_fn_3,'w') as export_script_i_f:
            #write the lines for PAM to read, using the above defined parameters, to import CAD (and mesh automatically) and build the tools
            str_i = str_ref_2.replace('$Simulation_file_i$',sample_path_3)
            str_i = str_i.replace('$MajorStrain_file_i$',majorStrain_path_3)
            str_i = str_i.replace('$MinorStrain_file_i$',minorStrain_path_3)
            str_i = str_i.replace('$MisesStress_file_i$',misesStress_path_3)
            str_i = str_i.replace('$Thinning_file_i$',thinning_path_3)
            str_i = str_i.replace('$DISPLACEMENT_file_i$',displacement_path_3)
            str_i = str_i.replace('$DEFORMED_MESH_file_i$',deformedMeshes_path_3)
            export_script_i_f.write(str_i)
        
        thinningContourExists = os.path.isfile(thinning_path_3)
        if thinningContourExists is not True: #only export the contours if they dont exist, if it does exists, don't generate
            
            #launch PAM to run its script to create the .mif mesh
            cmd='{} -play={}'.format(stamp_dn,export_script_i_fn_3)
            subprocess.call(cmd)
            
        print(simulation_folder, 'Stage III Completed')
        
        #create and write to a .txt file which will be the commands that PAM will use to import (and automatically mesh) the CAD files into the tool editor
        #this file contains scriptable commands which were obtained by recording them in PAM using the GUI
        export_script_i_fn_4 = PAMExportScripts_dir + 'ExportScript4_' + i_str + '.txt'
        
        with  open(export_script_i_fn_4,'w') as export_script_i_f:
            #write the lines for PAM to read, using the above defined parameters, to import CAD (and mesh automatically) and build the tools
            str_i = str_ref_2.replace('$Simulation_file_i$',sample_path_4)
            str_i = str_i.replace('$MajorStrain_file_i$',majorStrain_path_4)
            str_i = str_i.replace('$MinorStrain_file_i$',minorStrain_path_4)
            str_i = str_i.replace('$MisesStress_file_i$',misesStress_path_4)
            str_i = str_i.replace('$Thinning_file_i$',thinning_path_4)
            str_i = str_i.replace('$DISPLACEMENT_file_i$',displacement_path_4)
            str_i = str_i.replace('$DEFORMED_MESH_file_i$',deformedMeshes_path_4)
            export_script_i_f.write(str_i)
        
        thinningContourExists = os.path.isfile(thinning_path_4)
        if thinningContourExists is not True: #only export the contours if they dont exist, if it does exists, don't generate
            
            #launch PAM to run its script to create the .mif mesh
            cmd='{} -play={}'.format(stamp_dn,export_script_i_fn_4)
            subprocess.call(cmd)
            
        print(simulation_folder, 'Stage IV Completed')
            
    else: 
        print(simulation_folder, 'did not complete sucessfully')
        failedSamples.append(i_str)

print('### DONE ###')
        